## Prepare datasets

In [1]:
# from google.colab import drive

# drive.mount("/content/drive")

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
TRAIN_IMG_PATH = "../dataset/bootcamp/train/images"
TRAIN_MASK_PATH = "../dataset/bootcamp/train/annotations"
VAL_IMG_PATH = "../dataset/bootcamp/test/images"
VAL_MASK_PATH = "../dataset/bootcamp/test/annotations"

EDA_DS_PATH = "../dataset/bootcamp/train"
EDA_TRAIN_LABELS = os.path.join(EDA_DS_PATH, "train_labels.csv")
EDA_TRAIN_FILES = os.path.join(EDA_DS_PATH, "train_files.csv")
EDA_VAL_FILES = os.path.join(EDA_DS_PATH, "val_files.csv")

MODEL_PATH = "../models/UNet"

os.makedirs(MODEL_PATH, exist_ok=True)  # Ensure directory exist

In [4]:
files_df = pd.read_csv(EDA_TRAIN_FILES)
labels_df = pd.read_csv(EDA_TRAIN_LABELS)
val_files_df = pd.read_csv(EDA_VAL_FILES)

In [5]:
print(files_df.head())
print()
print(labels_df.head())
print()
train_files = files_df['filename'].to_numpy()
print(train_files[:5])
print()
val_files = val_files_df['filename'].to_numpy()
print(val_files[:5])

            filename
0  0001TP_006690.png
1  0001TP_006720.png
2  0001TP_006750.png
3  0001TP_006780.png
4  0001TP_006810.png

   class     label
0      0       sky
1      1  building
2      2      pole
3      3      road
4      4  sidewalk

['0001TP_006690.png' '0001TP_006720.png' '0001TP_006750.png'
 '0001TP_006780.png' '0001TP_006810.png']

['0016E5_07959.png' '0016E5_07961.png' '0016E5_07963.png'
 '0016E5_07965.png' '0016E5_07967.png']


## Import some Library

In [6]:
import torch
import torch.nn as nn
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import matplotlib.pyplot as plt

## Datasets Loader

In [7]:
# refer to dataset.py
from dataset import CityScapes

In [8]:
train_datasets = CityScapes(TRAIN_IMG_PATH, TRAIN_MASK_PATH, train_files)
image, mask = train_datasets.__getitem__(0)
print(image)
print()
print(mask)
print()
print(image.shape)
print()
print(mask.shape)

val_datasets = CityScapes(VAL_IMG_PATH, VAL_MASK_PATH, val_files)

tensor([[[-1.8439, -1.9124, -1.8782,  ..., -1.8268, -1.8610, -1.8268],
         [-1.8782, -1.8782, -1.8953,  ..., -1.8268, -1.8610, -1.8268],
         [-1.8953, -1.8610, -1.8610,  ..., -1.9124, -1.8782, -1.8439],
         ...,
         [-1.8610, -1.8610, -1.8439,  ..., -1.7240, -1.6727, -1.6727],
         [-1.8439, -1.8439, -1.8439,  ..., -1.6898, -1.6213, -1.7240],
         [-1.8439, -1.8610, -1.8439,  ..., -1.7240, -1.7240, -1.7412]],

        [[-1.7556, -1.8256, -1.7906,  ..., -1.6681, -1.7031, -1.7206],
         [-1.7906, -1.7906, -1.8081,  ..., -1.6681, -1.7031, -1.7206],
         [-1.8081, -1.7731, -1.7731,  ..., -1.7556, -1.7206, -1.7381],
         ...,
         [-1.7556, -1.7556, -1.7381,  ..., -1.4580, -1.4055, -1.3880],
         [-1.7381, -1.7381, -1.7381,  ..., -1.4580, -1.3704, -1.4405],
         [-1.7556, -1.7556, -1.7381,  ..., -1.4580, -1.4405, -1.4580]],

        [[-1.5256, -1.5953, -1.5604,  ..., -1.4210, -1.4559, -1.4384],
         [-1.5604, -1.5604, -1.5779,  ..., -1

In [9]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_datasets, batch_size=32, shuffle=True, num_workers=0)

## Model Architecture

In [10]:
from architecture import UNET

## Engine function

- Dice coefficient metrics

In [11]:
from engine import DiceLoss
from engine import DiceCrossEntropyLoss
from engine import train

Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 3\notebooks\engine.py:137: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  def train_engine(dataloader, model, loss_fn, optim, scaler=torch.cuda.amp.GradScaler()):


## Training with DiceLoss

In [14]:
model = UNET(in_channels=3, out_channels=len(labels_df))
model.cuda()
loss_fn = DiceLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)
print(model.out_channels)

12


In [15]:
model, logs = train(train_dataloader, val_dataloader, model, loss_fn, optim, 100, log_freq=1, save_best_model=True, 
              best_model_name='best_model_dice.pth', last_model_name='last_model_dice.pth', save_path=MODEL_PATH)

100%|██████████████████████████████████████████████████████████████████████| 12/12 [00:30<00:00,  2.50s/it, loss=0.871]
Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 3\notebooks\engine.py:154: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.91it/s, loss=0.975]


Epoch 0/99
----------
Train Loss: 0.8986, Val Loss: 0.9724
✅ Best model saved! (Val Loss: 0.9724295437335968)



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.95it/s, loss=0.857]


Epoch 1/99
----------
Train Loss: 0.8547, Val Loss: 0.8612
✅ Best model saved! (Val Loss: 0.8611787557601929)



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.99it/s, loss=0.819]


...

Epoch 97/99
----------
Train Loss: 0.5656, Val Loss: 0.7097



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.94it/s, loss=0.709]


Epoch 98/99
----------
Train Loss: 0.5646, Val Loss: 0.7101



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.94it/s, loss=0.703]

Epoch 99/99
----------
Train Loss: 0.5615, Val Loss: 0.7071



In [16]:
logs_df = pd.DataFrame(logs)
logs_df.to_csv(os.path.join(MODEL_PATH, 'logs_dice.csv'), index=False)

## Training with CrossEntropyLoss

In [17]:
model = UNET(in_channels=3, out_channels=len(labels_df))
model.cuda()
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)
print(model.out_channels)

12


In [18]:
model_2, logs_2 = train(train_dataloader, val_dataloader, model, loss_fn, optim, 100, log_freq=1, save_best_model=True, 
              best_model_name='best_model_cross.pth', last_model_name='last_model_cross.pth', save_path=MODEL_PATH)

100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.93it/s, loss=2.86]


Epoch 0/99
----------
Train Loss: 1.7162, Val Loss: 2.8588
✅ Best model saved! (Val Loss: 2.8588353991508484)



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.96it/s, loss=2.23]


Epoch 1/99
----------
Train Loss: 1.2231, Val Loss: 2.3103
✅ Best model saved! (Val Loss: 2.3103461861610413)



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.97it/s, loss=1.25]


...

Epoch 97/99
----------
Train Loss: 0.1364, Val Loss: 0.3382



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.91it/s, loss=0.324]


Epoch 98/99
----------
Train Loss: 0.1333, Val Loss: 0.3276



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  2.00it/s, loss=0.476]

Epoch 99/99
----------
Train Loss: 0.1405, Val Loss: 0.4245



In [19]:
logs_2_df = pd.DataFrame(logs_2)
logs_2_df.to_csv(os.path.join(MODEL_PATH, 'logs_cross.csv'), index=False)

## Training with DiceLoss combine with CrossEntropyLoss

In [12]:

model = UNET(in_channels=3, out_channels=len(labels_df))
model.cuda()
loss_fn = DiceCrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)
print(model.out_channels)

12


In [13]:
model_3, logs_3 = train(train_dataloader, val_dataloader, model, loss_fn, optim, 100, log_freq=1, save_best_model=True, 
              best_model_name='best_model_combined.pth', last_model_name='last_model_combined.pth', save_path=MODEL_PATH)

100%|███████████████████████████████████████████████████████████████████████| 12/12 [00:29<00:00,  2.47s/it, loss=1.14]
Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 3\notebooks\engine.py:183: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.94it/s, loss=2.46]


Epoch 0/99
----------
Train Loss: 1.3020, Val Loss: 2.4749
✅ Best model saved! (Val Loss: 2.474944770336151)



100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.96it/s, loss=2.33]


Epoch 1/99
----------
Train Loss: 0.9988, Val Loss: 2.3639
✅ Best model saved! (Val Loss: 2.3638995885849)



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.95it/s, loss=0.968]


...

Epoch 97/99
----------
Train Loss: 0.2094, Val Loss: 0.3675



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.90it/s, loss=0.392]


Epoch 98/99
----------
Train Loss: 0.2052, Val Loss: 0.3818



100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.86it/s, loss=0.368]

Epoch 99/99
----------
Train Loss: 0.2046, Val Loss: 0.3851



In [14]:
logs_3_df = pd.DataFrame(logs_3)
logs_3_df.to_csv(os.path.join(MODEL_PATH, 'logs_combined.csv'), index=False)